In [ ]:
# create composites 
# store as netcdf files
# v2: take all existing time steps at noon into account for a given storm, store integrated anomaly
#     store both an average and an integral for the variables it makes sense for 
#     integral for: chl, CO2 flux, NPP
#     all others: store avg only
#
# v3: like v2 but for new calculation of anomaly (subtract clim. first)
#
# HERE: corrected the identification of storms (previous version had ~50 duplicates in resulting files)
# fix: load all years at once and loop over the unique index_storm (as provided by txt file that contains the storms)


In [1]:
import os
from tqdm import tqdm
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from netCDF4 import Dataset, MFDataset
import numba as nb
import time as timing
from numba import njit 
from math import sin, cos, sqrt, atan2, radians
from geopy.distance import distance
import seawater as sw

In [2]:
#---
# FUNCTION 
#---

def get_distance_to_storm_center(lat2,lon2,aux_lat,aux_lon):
                
    # create list of locations within 1000km of the storm
    points_data = []
    for pp in range(0,lat2.shape[0]):
        aux = (lat2[pp],lon2[pp])
        points_data.append(aux)
        del aux

    #print(len(points_data))
    # for each of these points get the distance to the storm center in km -> get distance in x-dir and y-dir
    points_distance_x = np.zeros(len(points_data)) # distance in longitudinal direction, i.e., use latitude of storm (aux_lat)
    points_distance_y = np.zeros(len(points_data)) # distance in latitudinal direction, i.e., use longitude of storm (aux_lon)
    for pp in range(0,len(points_data)): 
        # distance in longitudinal direction
        aux_point = (aux_lat,points_data[pp][1])
        points_distance_x[pp] = distance(point_storm, aux_point).km
       # print(aux_point,point_storm,points_distance_x[pp])
        # check sign: if lon grid cell is smaller (=further west) than lon of storm, define distance to be negative
        if points_data[pp][1]<aux_lon:
            points_distance_x[pp] = -1*points_distance_x[pp]
        elif (aux_lon<0) & (points_data[pp][1]>0): # lon_storm is east of dateline, but grid cell is west of dateline (grid cell is also further west in this case!)
            if (points_data[pp][1]-360)<aux_lon:
                points_distance_x[pp] = -1*points_distance_x[pp]
        del aux_point
        # distance in latitudinal direction
        aux_point = (points_data[pp][0],aux_lon)
        points_distance_y[pp] = distance(point_storm, aux_point).km
        # check sign: if lat grid cell is smaller (=further south) than lat of storm, define distance to be negative
        if points_data[pp][0]<aux_lat:
            points_distance_y[pp] = -1*points_distance_y[pp]
        del aux_point  
    return points_distance_x,points_distance_y,points_data
                    
    
def bin_points_as_distance_to_storm_center(counter,points_distance_x,points_distance_y,x_bins,y_bins,aux_data_anom,data_storm_mean,data_storm_std,data_storm_count):
    # data_storm_mean,data_storm_std,data_storm_count: initialized arrays, will be filled in this function and then returned
    
    # bin the points (account for where each point is relative to storm center)
    ind_x = np.digitize(points_distance_x,x_bins,right=False) # minimum is 1 (not zero!!)
    ind_y = np.digitize(points_distance_y,y_bins,right=False)
 #   print(np.min(ind_x),np.max(ind_x))
 #   print(np.min(ind_y),np.max(ind_y))
    # returned index satisfies: bins[i-1] <= x < bins[i]

   # print(aux_data_anom.shape)
    for xx in range(1,len(x_bins)+1): # start at 1 here -> see note above for ind_x
        for yy in range(1,len(x_bins)+1):
            index = np.where((ind_y==yy) & (ind_x==xx))[0]
            if len(index)>0:
                #if counter==6: 
                #    print(xx,yy,index.shape,aux_data_anom.shape)
                #if (counter==40) & (xx==11) & (yy==20):
                #    print(xx,yy,len(points_distance_x))
                #    print(index)
                #    print(index.shape,aux_data_anom.shape)
                # anomaly 2
                data_storm_mean[xx-1,yy-1]  = np.nanmean(aux_data_anom[index])
                data_storm_std[xx-1,yy-1]   = np.nanstd(aux_data_anom[index])
                data_storm_count[xx-1,yy-1] = index.shape[0]
            del index
    return data_storm_mean,data_storm_std,data_storm_count
    

In [6]:
#---
# LOAD INFO FROM ALL YEARS AT ONCE
#----
import warnings
warnings.filterwarnings(action='ignore', message='Mean of empty slice')
    
save_netcdf = True
years = np.arange(1997,2018+1,1)

#vari_list = ['totChl','totChl_emulator','totChl_hr','MLD',\
#             'PAR_incoming','slp','photoC_total_surf']
#vari_list = ['wind_speed','diat_specific_growth_rate_surf','sp_specific_growth_rate_surf',\
#            'photoC_zint','phytoC_zint_100m','SST']

vari_list = ['totChl']
time_string = ''   #'_plus_2_days'

# for each year, find the number of storms
# go through storms and get avg/integral as a function of "distance to storm center"
# store the avg/integral anomaly for each storm
# also store: number of days the storm existed, avg SLP, min SLP

for vv in range(0,len(vari_list)):
    if vari_list[vv] in ['diat_specific_growth_rate_surf','mu_diat']:
        vari = 'mu_diat'
    elif vari_list[vv] in ['sp_specific_growth_rate_surf','mu_sp']:
        vari = 'mu_sp'
    elif vari_list[vv] in ['totChl_hr']:
        vari = 'totChl'
    else:
        vari = vari_list[vv] 
    #vari = vari_list[vv] #'totChl' # totChl, FG_CO2_2
    print('Process ',vari,time_string)    

    if vari in ['totChl']:
        long_name   = 'total chlorophyll'
        unit        = 'mg chl m-3'
    elif vari in ['totChl_emulator']:
        long_name   = 'total chlorophyll'
        unit        = 'mg chl m-3'
    elif vari in ['FG_CO2_2']:
        long_name   = 'air-sea CO2 flux'
        unit        = 'mmol m-3 cm s-1'
    elif vari in ['photoC_total_surf']:
        long_name   = 'total surface NPP'
        unit        = 'mmol m-3 cm s-1'
    elif vari in ['SST']:
        long_name   = 'Sea surface temperature (potential)'
        unit        = 'deg C'
    elif vari in ['MLD']:
        long_name   = 'mixed layer depth (density-based)'
        unit        = 'm' # actually in cm in file! convert further down!
    elif vari in ['wind_speed']:
        long_name   = '10m wind speed'
        unit        = 'm s-1'
    elif vari in ['diat_Fe_lim_surf']:
        long_name   = 'diatom surface iron limitation'
        unit        = 'n.d.'
    elif vari in ['diat_N_lim_surf']:
        long_name   = 'diatom surface nitrate limitation'
        unit        = 'n.d.'
    elif vari in ['diat_P_lim_surf']:
        long_name   = 'diatom surface phosphate limitation'
        unit        = 'n.d.'
    elif vari in ['diat_SiO3_lim_surf']:
        long_name   = 'diatom surface silicate limitation'
        unit        = 'n.d.'
    elif vari in ['sp_Fe_lim_surf']:
        long_name   = 'SP surface iron limitation'
        unit        = 'n.d.'
    elif vari in ['sp_N_lim_surf']:
        long_name   = 'SP surface nitrate limitation'
        unit        = 'n.d.'
    elif vari in ['sp_P_lim_surf']:
        long_name   = 'SP surface phosphate limitation'
        unit        = 'n.d.'
    elif vari in ['diatChl_SURF']:
        long_name   = 'diatom surface chlorophyll'
        unit        = 'mg chl m-3'
    elif vari in ['spChl_SURF']:
        long_name   = 'SP surface chlorophyll'
        unit        = 'mg chl m-3'
    elif vari in ['diat_light_lim_surf']:
        long_name   = 'diatom surface light limitation'
        unit        = 'n.d.'
    elif vari in ['sp_light_lim_surf']:
        long_name   = 'SP surface light limitation'
        unit        = 'n.d.'
    elif vari in ['diat_specific_growth_rate_surf','mu_diat']:
        long_name   = 'diatom surface specific growth rate'
        unit        = 'd-1'
    elif vari in ['sp_specific_growth_rate_surf','mu_sp']:
        long_name   = 'SP surface specific growth rate'
        unit        = 'd-1'
    elif vari in ['phytoC_zint_100m']:
        long_name   = 'total integrated phytoplankton C biomass in the top 100m'
        unit        = 'mmol C m-3'
    elif vari in ['photoC_zint']:
        long_name   = 'total vertically integrated NPP'
        unit        = 'mmol m-3 cm s-1 m2'
    elif vari in ['cloudfrac_isccp']:
        long_name   = 'ISCCP cloud cover'
        unit        = 'n.d.'
    elif vari in ['PAR_incoming']:
        long_name   = 'Incoming PAR (45% of incoming shortwave radiation)'
        unit        = 'W m-2'
    elif vari in ['slp']:
        long_name   = 'sea level pressure'
        unit        = 'Pa'
   
    if vari_list[vv] in ['totChl_hr']:
        path1 = '/global/cfs/cdirs/m4003/cnissen/CESM_anomalies_STORM_PAPER_subtract_clim_first/'+vari+'_hr_anomalies/'
    else:
        path1 = '/global/cfs/cdirs/m4003/cnissen/CESM_anomalies_STORM_PAPER_subtract_clim_first/'+vari+'_anomalies/'
        
    # where to save anomaly files?
    savepath     = path1 #'/global/cfs/cdirs/m4003/cnissen/CESM_'+vari+'_anomalies/'
    # check existence of paths
    if not os.path.exists(savepath):
        print ('Created '+savepath)
        os.makedirs(savepath)
        
    if vari_list[vv]=='totChl_hr':
        ds = xr.open_mfdataset(path1+'Anomalies_within_1000km_of_storm_center_'+vari+'_hr_JRA_grid_*noon'+\
                               time_string+'_subtract_clim_first.nc',\
                           concat_dim='count_anom',combine='nested')
    else:
        ds = xr.open_mfdataset(path1+'Anomalies_within_1000km_of_storm_center_'+vari+'_JRA_grid_*noon'+time_string+\
                               '_subtract_clim_first.nc',\
                           concat_dim='count_anom',combine='nested')
    print(ds['index_storm'])
    print()

    res    = 100
    x_bins = np.arange(-1000,1000+res,res)
    y_bins = np.arange(-1000,1000+res,res)

    dist_threshold = 1000

    # load lat/lon  
    if vari_list[vv] in ['totChl_hr']:
        file1 = 'Anomalies_within_1000km_of_storm_center_'+vari+'_hr_JRA_grid_1997-01-01_all_at_noon'+\
                        time_string+'_subtract_clim_first.nc'
    elif vari_list[vv] in ['totChl_emulator']:
        file1 = 'Anomalies_within_1000km_of_storm_center_'+vari+'_JRA_grid_1997-01-01_all_at_noon'+\
                        time_string+'_subtract_clim_first_full_field_clim.nc'
    else:
        file1 = 'Anomalies_within_1000km_of_storm_center_'+vari+'_JRA_grid_1997-01-01_all_at_noon'+\
                        time_string+'_subtract_clim_first.nc'
    ff2  = xr.open_dataset(path1+file1)
    lat       = ff2['lat'].values #[0:150] # model grid
    lon       = ff2['lon'].values # model grid
    lat,lon = np.meshgrid(lat,lon)
    lat = lat.transpose()
    lon = lon.transpose()
    ff2.close()

    #index_storm = np.asarray([int(x) for x in ds['index_storm']])
    list_storms = np.unique(ds['index_storm']) #np.asarray([int(x) for x in np.unique(ds['index_storm'])])
    num_storms = len(np.unique(ds['index_storm']))
    print('num_storms (ALL YEARS):',num_storms)

    #---
    # create netcdf file
    #---
    if save_netcdf:
        fv = -999
        if vari_list[vv] in ['totChl_hr']:
            netcdf_name = 'Composite_anomalies_within_'+str(dist_threshold)+'km_of_storm_center_at_noon_'+\
                                vari+'_hr_'+str(years[0])+'_'+str(years[-1])+time_string+'_subtract_clim_first_wind_speeds.nc'
        else:
            netcdf_name = 'Composite_anomalies_within_'+str(dist_threshold)+'km_of_storm_center_at_noon_'+\
                                vari+'_'+str(years[0])+'_'+str(years[-1])+time_string+'_subtract_clim_first_wind_speeds.nc'
        if not os.path.exists(savepath+netcdf_name):
            print('Create file '+savepath+netcdf_name)
            w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
            w_nc_fid.contact     = 'Cara Nissen, cara.nissen@colorado.edu'
            w_nc_fid.source_data = path1+file1
            w_nc_fid.script      = '/global/homes/c/cnissen/scripts/save_CESM_daily_chl_anomalies_composites_v3_subtract_clim_first_CORRECTED_max_wind_speeds_only.ipynb'
            w_nc_fid.sea_ice     = 'sea ice area is masked and not considered in composites'
            # create dimension & variable
            w_nc_fid.createDimension('x_bins', len(x_bins)) 
            w_nc_fid.createDimension('y_bins', len(y_bins)) 
            w_nc_fid.createDimension('num_storms', num_storms) 
            #w_nc_fid.createDimension('time', time_all[365:,:,:].shape[0]) 
      
            w_nc_var1 = w_nc_fid.createVariable('x_bins', 'f4',('x_bins'),fill_value=fv)
            w_nc_var1.description = 'Bins in x-direction (lon); '
            w_nc_var1 = w_nc_fid.createVariable('y_bins', 'f4',('y_bins'),fill_value=fv)
            w_nc_var1.description = 'Bins in y-direction (lat); '

            w_nc_var1 = w_nc_fid.createVariable('lon_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Longitude of storm center at min. SLP'
            w_nc_var1 = w_nc_fid.createVariable('lat_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Latitude of storm center at min. SLP'
            w_nc_var1 = w_nc_fid.createVariable('year_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Year of storm center at min. SLP'
            w_nc_var1 = w_nc_fid.createVariable('month_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Month of storm center at min. SLP'
            w_nc_var1 = w_nc_fid.createVariable('day_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Day of storm center at min. SLP'
            w_nc_var1 = w_nc_fid.createVariable('hour_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Hour of storm center at min. SLP'

            w_nc_var1 = w_nc_fid.createVariable('avg_max_wind_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Average max wind speed for each storm (averaged over each min. SLP at noon)'
            w_nc_var1.units = 'm s-1'
            w_nc_var1 = w_nc_fid.createVariable('max_max_wind_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Maximum maximum wind speed during the existence of each storm (based on min. SLP at noon)'
            w_nc_var1.units = 'm s-1'
                
            # write lat/lon to file
            w_nc_fid.variables['x_bins'][:]  = x_bins
            w_nc_fid.variables['y_bins'][:]  = y_bins

            w_nc_fid.close()

    # loop over storms
    counter = 0
    for ss in tqdm(list_storms): #range(0,num_storms)):

        ind_storm  = np.where(ds['index_storm']==ss)[0] 
        #print(ind_storm.shape[0])
        aux_array = ds['min_slp_storm'][ind_storm].values

        # additional information to be stored in file
        days_storm = ind_storm.shape[0] # duration of the storm -> to be stored in the file
        max_wind_avg_storm = np.mean(ds['max_wind_storm'][ind_storm]) # avg max. wind for all considered time steps -> to be stored in the file
        max_wind_max_storm = np.min(ds['max_wind_storm'][ind_storm]) # max. max. wind for all considered time steps -> to be stored in the fil
        
        if save_netcdf:
                
            w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
                        
            w_nc_fid.variables['avg_max_wind_storm'][counter] = max_wind_avg_storm
            w_nc_fid.variables['max_max_wind_storm'][counter] = max_wind_max_storm
                
            w_nc_fid.close()  
        
        counter = counter+1


Process  totChl 
<xarray.DataArray 'index_storm' (count_anom: 44258)>
dask.array<concatenate, shape=(44258,), dtype=float32, chunksize=(2126,), chunktype=numpy.ndarray>
Dimensions without coordinates: count_anom
Attributes:
    description:  index of storm (to know which storm imprints belong to the ...

num_storms (ALL YEARS): 9615


100%|██████████| 9615/9615 [05:08<00:00, 31.16it/s]


In [7]:
print(netcdf_name)

Composite_anomalies_within_1000km_of_storm_center_at_noon_totChl_1997_2018_subtract_clim_first_wind_speeds.nc
